In [1]:
from IPython.display import clear_output

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

import pandas as pd
import numpy as np

import json
import time
import mouse
import os
from os.path import exists
import re


In [3]:
# Подгрузка файла с ссылками нанужные страницы в авито (Собирали руками)
our_services = pd.read_excel("./data/our_services.xlsx", header = None)
our_links = our_services[7].dropna().unique().tolist()
len(our_links)

89

### Парсим последний уровень категоризатора по ссылкам, собранным на предыдущем этапе

In [6]:
# Подключаем драйвер
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

In [7]:
# Настраиваем костыльный счетчик
if os.path.exists('avito_counter.txt'):
    with open('avito_counter.txt') as f:
        j = int(f.readline())
else:
    j = 0
j

0

In [ ]:
# Количество страниц для парсинга 
pages = 4

# Имя файла с данными
file_name = 'data/avito_item_names_1.csv'

for url_temp in our_links[j:]:
    if 'avito' in url_temp:
        # Парсим выбранное количество страниц с данным фильтром
        for url in [f'{url_temp}?p={i}' for i in range(1, pages+1)]:
            driver.get(url)
    #         time.sleep(2)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)
            # Находим опорный элемент
            card_elements = driver.find_elements(by='xpath', value='//*[@itemprop="url"]/h3')
            card_name_list, card_main_price_list, extra_services_list = [], [], []

            # Парсим элементы со страницы
            for i, element in enumerate(card_elements):
                card_name = element.text
                card_main_price = element.find_element(by='xpath', value='../../../..')\
                                         .find_element(by='xpath', value='.//p[@data-marker="item-price"]').text
                extra_services = element.find_element(by='xpath', value='../../../..')\
                                        .find_elements(by='xpath', value='.//div[@class="price-list-value-bhOAA"]')
                extra_services = '; '.join([service.text for service in extra_services] if extra_services else [])\
                                    .replace('\n', '|')

                # Аппендим данные со страницы
                card_name_list.append(card_name)
                card_main_price_list.append(card_main_price)
                extra_services_list.append(extra_services)
                
            if not card_name_list:
                continue
                
            # Загружаем данные в csv
            pd.DataFrame(data = {'ind': j,
                                 'href': [url_temp]*len(card_name_list),
                                 'item_name': card_name_list,
                                 'value': card_main_price_list,
                                 'extra_services': extra_services_list}).to_csv(file_name, 
                                                                                 mode='a' if exists(file_name) else 'w', 
                                                                                 sep='\t', 
                                                                                 index=False,
                                                                                 header = not exists(file_name))

            print(j,
                  len(card_name_list), 
                  card_name_list[-1], 
                  card_main_price_list[-1], 
                  extra_services_list[-1], sep='\n')
            print()
    else:
        pass
    # Обновление счетчика
    j += 1
    with open('data/avito_counter.txt', 'w') as f:
        f.write(str(j))
    time.sleep(1)
    
else:
    # Удаление счетчика по окончании
    os.remove('data/avito_counter.txt')

In [11]:
# Ячейка для проверки

driver.get(url)
#         time.sleep(2)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(2)
# Находим опорный элемент
card_elements = driver.find_elements(by='xpath', value='//*[@itemprop="url"]/h3')
card_name_list, card_main_price_list, extra_services_list = [], [], []

# Парсим элементы со страницы
for i, element in enumerate(card_elements):
    card_name = element.text
    card_main_price = element.find_element(by='xpath', value='../../../..')\
                             .find_element(by='xpath', value='.//p[@data-marker="item-price"]').text
    extra_services = element.find_element(by='xpath', value='../../../..')\
                            .find_elements(by='xpath', value='.//div[@class="price-list-value-bhOAA"]')
    extra_services = '; '.join([service.text for service in extra_services] if extra_services else [])\
                        .replace('\n', '|')

    # Аппендим данные со страницы
    card_name_list.append(card_name)
    card_main_price_list.append(card_main_price)
    extra_services_list.append(extra_services)

print(j,
      len(card_name_list), 
      card_name_list[-1], 
      card_main_price_list[-1], 
      extra_services_list[-1], sep='\n')

0
50
Настройка и установка IP телефонии "Cisco"
от 10 000 ₽ за услугу
Подключение дачи, дома к интернету|от 5 000 ₽; Подключение спутникового интернета|от 5 000 ₽; Ещё 6 услуг
